1. `initialize_simulation()`: Starts up SUMO and initializes TraCI to connect to the SUMO simulation.

2. `get_av_ids()`: Retrieves the IDs of all autonomous vehicles (AVs) currently in the simulation using TraCI.

3. `get_distance_to_vehicle_in_front()`: Gets the distance from the AV to the vehicle in front of it using the TraCI `getLeader` function.

4. `get_distance_to_vehicle_behind()`: Gets the distance from the AV to the nearest vehicle behind it in the same lane within a 300m radius using TraCI context subscriptions. 

5. `calculate_safety_distances()`: Computes the minimum safe distances for crash avoidance (`dCrash`) and response (`dResponse`) based on the AV's speed and braking capabilities.

6. `check_lane_availability()`: Checks if the lanes to the left and right of the AV are available for a lane change maneuver.

7. `apply_lane_management_algorithm()`: Implements the decision making logic to compute the AV's new velocity and whether to change lanes based on the distances, speeds, lane availability etc.

8. `execute_vehicle_commands()`: Sets the AV's speed and performs a lane change in SUMO via TraCI if specified.

9. `manage_av()`: Main control loop function. Calls the above functions to percept the environment, make decisions and execute them. Runs each step for all AVs.

10. `run_simulation()`: Sets up and runs the SUMO simulation for a given duration, calling `manage_av()` at each time step.

In [1]:
import traci
import sys
import csv

`initialize_simulation()`: Starts up SUMO and initializes TraCI to connect to the SUMO simulation.
    
`Input`:
No input parameters.

`Process`:
1. Calls traci.start() to launch the TraCI client and connect it to a running SUMO simulation that is specified by the sumo command with configuration file.
2. The command launches SUMO with GUI mode and loads the simulation configuration from the file 'map2/osm.sumocfg'.
3. On successful connection it prints "TraCI started successfully".
4. If there is any error/exception in connecting to SUMO, it prints the error and exits the Python script.

`Output`:
No return value. Mainly sets up the connection to SUMO through TraCI which can then be used to control the simulation.

In [2]:
def initialize_simulation(sumocfg_file):
    try:
        traci.start(['sumo-gui', '-c', sumocfg_file]) 
        print("TraCI started successfully")
    except Exception as e:
        print("TraCI failed to start: ", e)
        sys.exit()

`get_av_ids()`: Retrieves the IDs of all autonomous vehicles (AVs) currently in the simulation.

`Input`: 
No input parameters.

`Process`:
1. Calls traci.vehicle.getIDList() to get a list of all vehicle IDs in the simulation.
2. Since this script is focused on controlling the autonomous vehicles (AVs), this function serves as a convenient way to retrieve IDs of all AVs currently loaded in the simulation.
3. The IDs can be used with TraCI to query state and control the AVs.

`Output`:
A list of strings containing vehicle IDs representing the AVs in the simulation.

In [3]:
def get_av_ids():
    return traci.vehicle.getIDList()

`get_distance_to_vehicle_in_front()`: Gets the distance from the AV to the vehicle in front of it using the TraCI `getLeader` function.

`Input`:
1. `av_id` - ID of the autonomous vehicle as string

`Process`:
1. Calls traci.vehicle.getLeader() to get info about the vehicle in front of the AV.
2. getLeader returns a tuple containing (vehicle ID, distance).
3. If no vehicle in front, it returns None.
4. Otherwise, the distance is extracted from the tuple.

`Output`:
1. Float value containing the distance in meters to the vehicle ahead.
2. If no vehicle, returns float('inf') i.e. infinity.

In [4]:
def get_distance_to_vehicle_in_front(av_id):
    leader_info = traci.vehicle.getLeader(av_id)
    if leader_info is not None:
        _, distance = leader_info
        return distance
    else:
        return float('inf')

`get_distance_to_vehicle_behind()`: Gets the distance from the AV to the nearest vehicle behind it in the same lane within a 300m radius using TraCI context subscriptions.

`Input`:
1. `av_id` - ID of the autonomous vehicle as string

`Process`:
1. Get AV's current position and lane ID using TraCI
2. Subscribe to context information of all vehicles within 300m radius using TraCI API.
3. Context includes position and lane ID of vehicles.
4. Check if vehicle is in same lane as AV and is behind the AV based on position.
5. Compute distance between those vehicles and AV using TraCI simulation.getDistance2D()
6. Return minimum distance across all vehicles behind AV in same lane.

`Output`:
1. Float value with distance in meters to nearest vehicle behind in same lane, within 300m.
2. None if no vehicle found behind in same lane within 300m.

In [5]:
def get_distance_to_vehicle_behind(av_id):
    radius = 1000
    av_position = traci.vehicle.getPosition(av_id)
    av_lane = traci.vehicle.getLaneID(av_id)
    traci.vehicle.subscribeContext(av_id, traci.constants.CMD_GET_VEHICLE_VARIABLE, \
        radius, [traci.constants.VAR_POSITION, traci.constants.VAR_LANE_ID])
    context = traci.vehicle.getContextSubscriptionResults(av_id)
    min_distance = float('inf')

    if context:
        for vehicle_id, vehicle_data in context.items():
            if vehicle_id != av_id and vehicle_data[traci.constants.VAR_LANE_ID] == av_lane:
                vehicle_position = vehicle_data[traci.constants.VAR_POSITION]
                distance = traci.simulation.getDistance2D(av_position[0], av_position[1], \
                    vehicle_position[0], vehicle_position[1])
                if 0 < distance < min_distance:
                    min_distance = distance

    return min_distance if min_distance != float('inf') else None

`calculate_safety_distances()`: Computes the minimum safe distances for crash avoidance (`dCrash`) and response (`dResponse`) based on the AV's speed and braking capabilities.

`Input`:
1. `av_id` - ID of the autonomous vehicle as string

`Process`:
1. Get AV's current speed using TraCI
2. Define constant values for reaction time, comfortable braking rate, max braking rate
3. Apply dynamic vehicle model equations to calculate:
4. dCrash: Minimum distance to stop before crashing in case of sudden deceleration of front vehicle
5. dResponse: Minimum distance to begin responding to avoid getting too close

`Output`:
1. `dCrash` - Float value containing safe distance in m to avoid crash
2. `dResponse` - Float value containing safe distance in m to trigger response

In [6]:
def calculate_safety_distances(av_id):
    reaction_time = 1.5
    comfortable_braking = traci.vehicle.getDecel(av_id) 
    max_braking = traci.vehicle.getEmergencyDecel(av_id)
    current_speed = traci.vehicle.getSpeed(av_id)
    
    dCrash = current_speed * reaction_time + (1/2 * comfortable_braking * reaction_time**2) \
             + (current_speed + reaction_time * comfortable_braking)**2 / (2 * max_braking)
    
    dResponse = current_speed * reaction_time + (1/2 * comfortable_braking * reaction_time**2) \
                + (current_speed + reaction_time * comfortable_braking)**2 / (2 * comfortable_braking)

    return dCrash, dResponse

`check_lane_availability()`: Checks if the lanes to the left and right of the AV are available for a lane change maneuver.

`Input`:
1. `av_id`: The ID of the autonomous vehicle (AV) for which lane availability is being checked.

`Process`:
1. First, the function calculates safety distances using `calculate_safety_distances`.
2. It checks if the AV can change to the left or right lane using `traci.vehicle.couldChangeLane`.
3. If a lane change is feasible, it assesses the safety of this maneuver by calling `check_lane_safety`.

`Output`:
1. A dictionary indicating the availability of left and right lanes for safe lane changes, with boolean values.

---

`check_lane_safety()`

`Input`:
1. `av_id`: The ID of the autonomous vehicle.
2. `lane_offset`: The direction and magnitude of the lane change (e.g., -1 for left, 1 for right).
3. `dResponse`: A safety distance threshold.

`Process`:
1. Identifies the target lane using `get_target_lane_id`.
2. Checks for other vehicles in the target lane and their distances to the AV.
3. Determines if the lane change is safe based on the proximity of other vehicles within the `dResponse` distance.

`Output`:
1. A boolean value indicating whether the lane change to the target lane is safe.

---

`get_target_lane_id()`

`Input`:
1. `av_id`: The ID of the autonomous vehicle.
2. `lane_offset`: The direction and magnitude of the lane change (e.g., -1 for left, 1 for right).

Process:
1. Calculates the current and target lane indices based on the AV's position and the lane change direction.
2. Constructs the target lane ID using the current road ID and the target lane index.
3. Ensures the target lane index is within the valid range for the current road.

`Output`:
1. The ID of the target lane if the lane change is within the valid range, otherwise `None`.

---

These functions work together in an autonomous vehicle simulation environment to evaluate and ensure the safety and feasibility of lane changes.

In [7]:
def check_lane_availability(av_id):
    lane_availability = {'left': False, 'right': False}
    _, dResponse = calculate_safety_distances(av_id)

    if traci.vehicle.couldChangeLane(av_id, -1):
        if check_lane_safety(av_id, -1, dResponse):
            lane_availability['left'] = True

    if traci.vehicle.couldChangeLane(av_id, 1):
        if check_lane_safety(av_id, 1, dResponse):
            lane_availability['right'] = True

    return lane_availability

def check_lane_safety(av_id, lane_offset, dResponse):
    target_lane_id = get_target_lane_id(av_id, lane_offset)
    if not target_lane_id:
        return False

    vehicles_in_target_lane = traci.lane.getLastStepVehicleIDs(target_lane_id)
    av_position = traci.vehicle.getPosition(av_id)

    for vehicle_id in vehicles_in_target_lane:
        vehicle_position = traci.vehicle.getPosition(vehicle_id)
        distance = traci.simulation.getDistance2D(av_position[0], av_position[1], vehicle_position[0], vehicle_position[1])

        if distance < dResponse:
            return False

    return True

def get_target_lane_id(av_id, lane_offset):
    current_lane_index = traci.vehicle.getLaneIndex(av_id)
    target_lane_index = current_lane_index + lane_offset
    edge_id = traci.vehicle.getRoadID(av_id)

    current_lane_id = traci.vehicle.getLaneID(av_id)
    num_lanes = current_lane_id.count(':') + 1

    if 0 <= target_lane_index < num_lanes:
        target_lane_id = edge_id + '_' + str(target_lane_index)
        return target_lane_id
    else:
        return None

`apply_lane_management_algorithm()`: Implements the decision making logic to compute the AV's new velocity and whether to change lanes based on the distances, speeds, lane availability etc.

`Input`:
1. Distances to front and rear vehicles
2. Precomputed safe distances
3. Current velocity and speed limit
4. Boolean flags indicating lane availability

`Process`:
1. If distance to front vehicle < minimum crash avoidance distance, brake completely to 0 velocity
2. Else if distance < minimum response distance, decelerate by 5 m/s
3. If unsafe distance and lane change possible, pick left or right lane
4. Return new velocity and target lane for lane change

`Output`:
1. `newVelocity` - Updated velocity value
2. `laneChange` - Lane offset (-1 or 1) to change lane, None if no change    

In [8]:
def apply_lane_management_algorithm(dFront, dBehind, dCrash, dResponse, vCurrent, vLimit, laneAvailability):
    newVelocity = vCurrent
    laneChange = None

    if dFront < dCrash:
        newVelocity = 0
    elif dFront < dResponse:
        newVelocity = max(vCurrent - 5, 0)

    if dFront < dResponse and (laneAvailability['left'] or laneAvailability['right']):
        laneChange = -1 if laneAvailability['left'] else 1

    return newVelocity, laneChange

`execute_vehicle_commands()`: Sets the AV's speed and performs a lane change in SUMO via TraCI if specified.

`Input`:
1. `av_id` - ID of the autonomous vehicle as string
2. `newVelocity` - Updated velocity value computed by algorithm
3. `laneChange` - Lane offset (-1, 0 or +1) to change lane, None if no change

`Process`:
1. Calls traci.vehicle.setSpeed() to update AV's velocity
2. If laneChange is not None, calls traci.vehicle.changeLane() to initiate lane change maneuver over 10 seconds duration

`Output`:
1. No return value. Applies commands computed by decision algorithm to vehicle in SUMO simulation using TraCI API.

In [9]:
def execute_vehicle_commands(av_id, newVelocity, laneChange):
    traci.vehicle.setSpeed(av_id, newVelocity)
    
    if laneChange is not None:
        traci.vehicle.changeLane(av_id, laneChange, duration=7.5)  # Change over 10 seconds

`manage_av()`: Main control loop function. Calls other functions to percept environment, make decisions and execute. 

`Input`: 
1. `av_id` - ID of the autonomous vehicle 

`Process`:
1. Get distances to front and rear vehicles using corresponding functions
2. Compute safe distances based on speed
3. Check if lane changes are possible on left and right
4. Call algorithm to calculate new velocity and lane change offset
5. Execute updated velocity and lane change using TraCI

`Output`: 
1. No return value. Controls the AV using TraCI by going through one iteration of the sense-plan-act loop.

This function serves as the main wrapper that executes the entire autonomous driving logic for one AV at one time step.

In [10]:
def manage_av(av_id):
    dFront = get_distance_to_vehicle_in_front(av_id)
    dBehind = get_distance_to_vehicle_behind(av_id)
    dCrash, dResponse = calculate_safety_distances(av_id)
    vCurrent = traci.vehicle.getSpeed(av_id)
    vLimit = traci.vehicle.getMaxSpeed(av_id)
    laneAvailability = check_lane_availability(av_id)
    newVelocity, laneChange = apply_lane_management_algorithm(
        dFront, dBehind, dCrash, dResponse, vCurrent, vLimit, laneAvailability
    )
    execute_vehicle_commands(av_id, newVelocity, laneChange)

In [11]:
def close_simulation():
    print("Closing TraCI connection")
    traci.close()
    print("TraCI connection closed")

`run_simulation()`: Sets up and runs the SUMO simulation, executing AV control loop at each time step.

`Input`: No input parameters.

`Process`:
1. Initialize TraCI client and connect to SUMO 
2. Get list of AV IDs to control
3. Set simulation duration  
4. Run control loop while SUMO is simulating
    - Step simulation 1 step at a time
    - Set color for AVs 
    - For each AV:
        - Get environment data  
        - Compute decisions
        - Execute using TraCI
    - Simulation step
5. Close TraCI connection when done

`Output`: No return value. Runs the SUMO simulation and controls AVs using TraCI.

This function brings together all the components to setup the simulation, perceive environment, run control loop for each AV and simulate traffic.

In [12]:
# def run_simulation():
#     sumocfg_file = 'map2\osm.sumocfg'
#     initialize_simulation(sumocfg_file)
#     av_ids = ['veh10', 'veh20', 'veh30', 'veh40', 'veh50', 'veh60','veh70']
#     simulation_duration = 200

#     try:
#         while traci.simulation.getMinExpectedNumber() > 0:
#             if traci.simulation.getTime() >= simulation_duration:
#                 break

#             traci.simulationStep()
#             for av_id in av_ids:
#                 if av_id in traci.vehicle.getIDList():
#                     traci.vehicle.setColor(av_id, (255, 0, 0))  # Set AVs color to red

#             traci.simulationStep()
#             for av_id in get_av_ids():
#                 if av_id in av_ids:
#                     dFront = get_distance_to_vehicle_in_front(av_id)
#                     dBehind = get_distance_to_vehicle_behind(av_id)
#                     dCrash, dResponse = calculate_safety_distances(av_id)
#                     vCurrent = traci.vehicle.getSpeed(av_id)
#                     vLimit = traci.vehicle.getMaxSpeed(av_id)
#                     laneAvailability = check_lane_availability(av_id)

#                     newVelocity, laneChange = apply_lane_management_algorithm(
#                         dFront, dBehind, dCrash, dResponse, vCurrent, vLimit, laneAvailability
#                     )

#                     execute_vehicle_commands(av_id, newVelocity, laneChange)

#                     print(f"AV {av_id} - Time: {traci.simulation.getTime()}, dFront: {dFront}, dBehind: {dBehind}, "
#                         f"dCrash: {dCrash}, dResponse: {dResponse}, Velocity: {vCurrent}->{newVelocity}, LaneChange: {laneChange}")

#         close_simulation()

#     except Exception as e:
#         print("Error during simulation: ", e)
#         traci.close()
#         sys.exit()

# if __name__ == "__main__":
#     run_simulation()

In [13]:
def run_simulation():
    sumocfg_file = 'map2\osm.sumocfg'
    initialize_simulation(sumocfg_file)
    av_ids = ['veh10', 'veh20', 'veh30', 'veh40', 'veh50', 'veh60','veh70']
    simulation_duration = 200

    try:
        with open('simulation_output.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            # Write CSV header
            writer.writerow(['AV ID', 'Time', 'dFront', 'dBehind', 'dCrash', 'dResponse', 'Current Velocity', 'New Velocity', 'Lane Change'])

            while traci.simulation.getMinExpectedNumber() > 0:
                if traci.simulation.getTime() >= simulation_duration:
                    break

                traci.simulationStep()
                for av_id in av_ids:
                    if av_id in traci.vehicle.getIDList():
                        traci.vehicle.setColor(av_id, (255, 0, 0))  # Set AVs color to red

                traci.simulationStep()
                for av_id in get_av_ids():
                    if av_id in av_ids:
                        dFront = get_distance_to_vehicle_in_front(av_id)
                        dBehind = get_distance_to_vehicle_behind(av_id)
                        dCrash, dResponse = calculate_safety_distances(av_id)
                        vCurrent = traci.vehicle.getSpeed(av_id)
                        vLimit = traci.vehicle.getMaxSpeed(av_id)
                        laneAvailability = check_lane_availability(av_id)

                        newVelocity, laneChange = apply_lane_management_algorithm(
                            dFront, dBehind, dCrash, dResponse, vCurrent, vLimit, laneAvailability
                        )

                        execute_vehicle_commands(av_id, newVelocity, laneChange)

                        # Write data to CSV file
                        writer.writerow([av_id, traci.simulation.getTime(), dFront, dBehind, dCrash, dResponse, vCurrent, newVelocity, laneChange])

        close_simulation()

    except Exception as e:
        print("Error during simulation: ", e)
        traci.close()
        sys.exit()

if __name__ == "__main__":
    run_simulation()

TraCI started successfully
Closing TraCI connection
TraCI connection closed
